# Nonlinear

## Import modules

In [1]:
# Basic stuff
import sys
sys.path.append("/net/server/homes/sep/gbarnier/code/gpu/acousticIsoOp/test/lib/python/")
import genericIO
import SepVector
import Hypercube
import Acoustic_iso_double
import numpy as np
import time

# Plotting library
import matplotlib.pyplot as plt
import sepPlot


## Wavelet

In [2]:
!/net/server/homes/sep/gbarnier/code/gpu/acousticIsoOp/test/bin/waveletMain.py nts=1000 dts=0.008 timeDelay=1.0 f1=2 f2=5 f3=8 f4=15 type=ali wavelet=wlt.H
!Spectra < wlt.H | Graph grid=y title="Wavelet spectrum" max1=30 label1="Frequency [Hz]" | Xtpen 
!Graph grid=y max1=2.0 title="Wavelet" < wlt.H | Xtpen 

## Velocity model

In [3]:
!/net/server/homes/sep/gbarnier/code/gpu/acousticIsoOp/test/bin/padFileGpuMain zPad=100 xPad=100 model=velocityMarmousi.H data=velocityMarmousi.pad.H

 
------------------------ Model padding program --------------------
Original nz = 350 [samples]
Original nx = 1700 [samples]
 
zPadMinus = 100 [samples]
zPadPlus = 110 [samples]
xPadMinus = 100 [samples]
xPadPlus = 104 [samples]
 
blockSize = 16 [samples]
FAT = 5 [samples]
 
New nz = 570 [samples including padding and FAT]
New nx = 1914 [samples including padding and FAT]
-------------------------------------------------------------------
 


## Initialize nonlinear operator

In [4]:
# Note: you need a "dummy arg" for the first argument to replace what would be the executable
args=["dummy arg","vel=velocityMarmousi.pad.H","model=wlt.H","par=parPythonTest.p"]
modelDouble, dataDouble, velDouble, parObject, sourcesVector, receiversVector = Acoustic_iso_double.nonlinearOpInitDouble(args)

# Printing parameter file for reference
!cat parPythonTest.p


nts=1000
dts=0.008
sub=8
nz=570
nx=1914
zPadMinus=100
zPadPlus=110
xPadMinus=100
xPadPlus=104
dz=0.01
dx=0.01
fMax=20
zSource=2
xSource=1
nShot=170
spacingShots=10
depthReceiver=2
nReceiver=1700
dReceiver=1
oReceiver=1
saveWavefield=0
wavefieldShotNumber=0
blockSize=16
fat=5
nGpu=8


## Create nonlinear object

In [5]:
nonlinearOp=Acoustic_iso_double.nonlinearPropShotsGpu(modelDouble,dataDouble,velDouble,parObject,sourcesVector,receiversVector)

## Read model (wavelet)

In [6]:
modelFile=parObject.getString("model")
modelFloat=genericIO.defaultIO.getVector(modelFile,ndims=3)
modelDMat=modelDouble.getNdArray()
modelSMat=modelFloat.getNdArray()
modelDMat[:]=modelSMat

## Nonlinear forward

In [7]:
start=time.time()
nonlinearOp.forward(False,modelDouble,dataDouble);
end=time.time()
print("Computational time for forward = ",end-start," [s]")

Computational time for forward =  34.09494185447693  [s]


In [8]:
print(dataDouble.getHyper().axes[2].o)

2.421438694000244e-08


## Nonlinear adjoint

In [9]:
# Create a model vector for adjoint result
modelDoubleAdj=SepVector.getSepVector(modelDouble.getHyper(),storage="dataDouble")

# Apply adjoint
start=time.time()
nonlinearOp.adjoint(False,modelDoubleAdj,dataDouble);
end=time.time()
print("Computational time for adjoint = ",t1-t0," [s]")

NameError: name 't1' is not defined

## Display results 
help(sepPlot) to access self documention

In [10]:
help(sepPlot)

Help on module sepPlot:

NAME
    sepPlot

CLASSES
    builtins.object
        cubePlot
        cubeplotStatic
        sepPlot
            plot1D
                Dots
                Graph
            plot2D
                Contour
                Grey
    
    class Contour(plot2D)
     |  Generic plot class that will setup good defaults
     |  
     |  Method resolution order:
     |      Contour
     |      plot2D
     |      sepPlot
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __init__(self, plt, data, **kw)
     |      Initialize sePlot 
     |      plt - matplotlib.plot object
     |      data - sepVector object
     |      kw   - optional arguments
     |  
     |  output(self)
     |      Output figure
     |  
     |  setDefaults(self)
     |      setDefaults for contour
     |  
     |  ----------------------------------------------------------------------
     |  Methods inherited from plot2D:
     |  
     |  orrientArray(self, mat)
 

### Model

In [ ]:
# Convert model to 1D float
modelFloat1D=SepVector.getSepVector(Hypercube.hypercube(axes=[modelDoubleAdj.getHyper().axes[0]]))
modelFloat1DNd=modelFloat1D.getNdArray()
modelDoubleNd=modelDouble.getNdArray()
modelFloat1DNd[:]=modelDoubleNd

# Plot output model after adjoint
sepPlot.Graph(plt,modelFloat1D,label1="Time [s]",label2="Amplitude",title="Adjoint").output()

### Forward

In [ ]:
# Convert data to 2D float 
dataFloat2D=SepVector.getSepVector(Hypercube.hypercube(axes=[dataDouble.getHyper().axes[0],dataDouble.getHyper().axes[1]]))
dataFloat2DNd=dataFloat2D.getNdArray()
dataDoubleNd=dataDouble.getNdArray()
dataFloat2DNd[:]=dataDoubleNd

# Plot shot gather
sepPlot.Grey(plt,dataFloat2D,label1="Time [s]",label2="Receivers [km]",title="Forward").output()

### Adjoint

In [ ]:
# Convert model to 1D float
adjointFloat1D=SepVector.getSepVector(Hypercube.hypercube(axes=[modelDoubleAdj.getHyper().axes[0]]))
adjointFloat1DNd=adjointFloat1D.getNdArray()
modelDoubleNd=modelDoubleAdj.getNdArray()
adjointFloat1DNd[:]=modelDoubleNd

# Plot output model after adjoint
sepPlot.Graph(plt,adjointFloat1D,label1="Time [s]",label2="Amplitude",title="Adjoint").output()

## Write results to disk

In [ ]:
# Data 
dataFloat=SepVector.getSepVector(dataDouble.getHyper(),storage="dataFloat")
dataFloatNp=dataFloat.getNdArray()
dataDoubleNp=dataDouble.getNdArray()
dataFloatNp[:]=dataDoubleNp
_=genericIO.defaultIO.writeVector("nonlinearFwdJupyter.H",modelFloat)

# Model after applying FWD and ADJ
modelFloat=SepVector.getSepVector(modelDoubleAdj.getHyper(),storage="dataFloat")
modelFloatNp=modelDoubleAdj.getNdArray()
modelDoubleNp=modelDoubleAdj.getNdArray()
modelFloatNp[:]=modelDoubleNp
_=genericIO.defaultIO.writeVector("nonlinearAdjJupyter.H",modelFloat)

## Dot product test

In [ ]:
propagator.dotTest(True)